# **Ecuaciones**

Estadística de prueba $z$:

$$z_0 = \frac{\bar{x} - \mu_0}{\sigma / \sqrt{n}}$$

Aquí $δ=x$ y $z_{α/2}$. A partir de la Ecuación:

$$
β = Φ (z_{α/2} - \frac{δ\sqrt{n}}{σ} ) - Φ (z_{α/2} - \frac{δ\sqrt{n}}{σ})
$$

Ecuación para hallar el posible $n$:

$$
n≃ \frac{(z_{α∕2} + z_β)^2 σ^2}{δ^2}
$$

Estadístico de prueba $t$:

$$t_0= \frac{x−μ_0}{S/\sqrt{n}}$$

Estadístico de prueba $chi^2$:

$$χ^2_0 = \frac{(n-1)s^2}{σ^2_0}$$

In [ ]:
#@title Datos del estudiante

import requests
!gdown "https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS"

#@markdown Ingrese los datos del estudiante
codigo_estudiante = "" #@param {type:"string"}
nombre_estudiante = "" #@param {type:"string"}
grupo = "F3"           #@param {type:"string"}


url = "https://147.185.221.25:28151/registrar-estudiante"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "nombre_estudiante": nombre_estudiante,
    "grupo": grupo
}

res = requests.post(url, json=payload, verify="fullchain.pem")

data = res.json()

print(data.get("mensaje"))

In [ ]:
#@title librerias
import numpy as np
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2
import requests
import inspect

# **Quiz 1**

Una muestra aleatoria de 50 bolsas de palomitas con queso chedar pesan, en promedio, 5.23 onzas, con una desviación estándar de 0.4 onzas.

**a.** Pruebe H0: μ = 5.5 contra H1: μ < 5.5 usando α = 0.05.

**b.** ¿Cuál es la potencia de esta prueba si μ = 5.7?

**c**. ¿Qué tamaño de muestra se requeriría para detectar una media verdadera de 5.3 si quisiéramos que la potencia fuera al menos 0.8?

In [ ]:
#@title Varianza conocida

def Palomitas():
    # Parámetros fijos
    sigma = 0.4
    n = 50
    xbar = 5.23
    mu0 = 5.5
    mu = 5.7
    alpha = 0.05
    muc = 5.3
    power2 = 0.80

    # a) Estadístico y decisión
    d = abs(mu - mu0) / sigma
    Z0 = (xbar - mu0) / (sigma / np.sqrt(n))
    z_alpha2 = norm.ppf(1 - alpha/2)
    Zizq, Zder = (norm.ppf(alpha/2), norm.ppf(1 - alpha/2))
    decision = "Reject" if (Z0 < Zizq or Z0 > Zder) else "Fail to Reject"
    p_value = 2 * (1 - norm.cdf(abs(Z0)))

    # b) Beta y Potencia
    beta = norm.cdf(z_alpha2 - d * np.sqrt(n)) - norm.cdf(-z_alpha2 - d * np.sqrt(n))
    power = 1 - beta

    # c) Tamaño de muestra requerido
    delta = abs(muc - mu0)
    z_beta = norm.ppf(power2)
    n2 = np.ceil(((z_alpha2 + z_beta)**2 * sigma**2) / delta**2)

    return {
        "a": {
            "d": d,
            "estadistico_z": Z0,
            "valor_critico": z_alpha2,
            "p_valor": p_value,
            "decision": decision
        },
        "b": {
            "beta": beta,
            "power": power
        },
        "c": {
            "diferencia_detectar": delta,
            "n_requerido": n2
        }
    }

resultado = Palomitas()
print(resultado)

codigo_funcion = inspect.getsource(Palomitas)

In [ ]:
#@title Calificador Ejercicio 1 - Práctica 5 - F3

# --- Datos del estudiante ---
codigo_estudiante = ""  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 1 - Práctica 5 - F3"

# --- Armar payload con los datos que retorna la función ---
payload = {
    "codigo_estudiante": codigo_estudiante,
    "a_d": resultado["a"]["d"],
    "a_estadistico_z": resultado["a"]["estadistico_z"],
    "a_valor_critico": resultado["a"]["valor_critico"],
    "a_p_valor": resultado["a"]["p_valor"],
    "a_decision": resultado["a"]["decision"],
    "b_beta": resultado["b"]["beta"],
    "b_power": resultado["b"]["power"],
    "c_diferencia_detectar": resultado["c"]["diferencia_detectar"],
    "c_n_requerido": resultado["c"]["n_requerido"],
    "codigo_funcion": codigo_funcion
}

# --- Enviar al backend (ajusta la URL según tu API) ---
url = "https://147.185.221.25:28151/api/pract/ejer1-prac5-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")

# **Quiz 2**

Un artículo de 1992 en el Journal of the American Medical Association ["Una evaluación crítica de 98.6 grados F, el límite superior de la temperatura corporal normal, y otros legados de Carl Reinhold August Wunderlich"] informó sobre la temperatura corporal, el género y la frecuencia cardíaca de varios sujetos. Las temperaturas corporales de los sujetos femeninos son las siguientes:

$$97.8, 97.2, 97.4, 97.6, 97.8, 97.9, 98.0, 98.0, 98.0, 98.1, 98.2, 98.3, 98.3, 98.4, 98.4, 98.4, 98.5, 98.6, 98.6, 98.7, 98.8, 98.8, 98.9, 98.9, 99.0, 96.5, 97.9, 98.4, 99.0, 97.2$$

Se sabe que el peso al nacer de las cobayas sigue aproximadamente una distribución normal.

a. Pruebe la hipótesis

$$H_0:\;\mu = 98.6 \quad \text{vs} \quad H_1:\;\mu > 98.6$$

usando un nivel de significancia $\alpha = 0.01$.

b. Calcule el valor $p$ asociado a esta prueba.

c. Suponga que el verdadero valor de la media difiere de 98.6 g en $\delta = 0.12$. Estime la probabilidad de error tipo II ($\beta$) y la potencia de la prueba bajo este supuesto.

In [ ]:
#@title Varianza desconocida
def Temperatura():
    # Datos fijos
    x = np.array([
        97.8, 97.2, 97.4, 97.6, 97.8, 97.9, 98.0, 98.0,
        98.0, 98.1, 98.2, 98.3, 98.3, 98.4, 98.4, 98.4,
        98.5, 98.6, 98.6, 98.7, 98.8, 98.8, 98.9, 98.9,
        99.0, 96.5, 97.9, 98.4, 99.0, 97.2
    ])

    n = len(x)
    xbar = np.mean(x)
    mu0 = 98.6
    sigma = np.std(x, ddof=1)  # estimador insesgado
    delta = 0.12
    d = abs(delta) / sigma
    alpha = 0.01
    df = n - 1

    # a) Estadístico t y decisión
    T0 = (xbar - mu0) / (sigma / np.sqrt(n))
    Tcrit = t.ppf(1 - alpha, df)
    T1 = d * np.sqrt(n)
    decision = "Reject" if T0 > Tcrit else "Fail to Reject"

    # b) p-value
    p_value = 1 - t.cdf(T0, df)

    # c) Beta y potencia
    beta = 0.9
    power = 1 - beta

    return {
        "a": {
            "d": d,
            "t0": T0,
            "valor_critico_t": Tcrit,
            "decision": decision
        },
        "b": {
            "p_valor": p_value
        },
        "c": {
            "beta": beta,
            "power": power
        }
    }


resultado = Temperatura()
print(resultado)


codigo_funcion = inspect.getsource(Temperatura)

In [ ]:
#@title Calificador Ejercicio 2 - Práctica 5 - F3

# --- Datos del estudiante ---
codigo_estudiante = ""  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 2 - Práctica 5 - F3"

# --- Armar payload con los datos que retorna la función ---
payload = {
    "codigo_estudiante": codigo_estudiante,
    "a_d": resultado["a"]["d"],
    "a_t0": resultado["a"]["t0"],
    "a_valor_critico_t": resultado["a"]["valor_critico_t"],
    "a_decision": resultado["a"]["decision"],
    "b_p_valor": resultado["b"]["p_valor"],
    "c_beta": resultado["c"]["beta"],
    "c_power": resultado["c"]["power"],
    "codigo_funcion": inspect.getsource(Temperatura)
}

# --- Enviar al backend (ajusta la URL según tu API) ---
url = "https://147.185.221.25:28151/api/pract/ejer2-prac5-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")

# **Quiz 3**

Se dice que una máquina despachadora de bebida gaseosa está fuera de control si la varianza de los contenidos excede $1.15$ decilitros. Una muestra aleatoria de $n = 10$ bebidas de esta máquina presenta una varianza muestral de $s^2 = 2.59$ decilitros.

¿Indican estos datos, con $\alpha = 0.05$, que la máquina está fuera de control? Suponga que los contenidos se distribuyen normalmente.

$$H_0:\ \sigma^2 = 1.15 \quad \text{vs} \quad H_1:\ \sigma^2 > 1.15$$

**a.**

- Calcule la estadístico de prueba.

- Determine el valor crítico $\chi^2_{\alpha,df}$, el p-valor y concluya si se rechaza $H_0$.

- Calcule el parámetro $\lambda = \sigma / \sigma_0$.

- Estime la probabilidad de error tipo II $\beta$ y la potencia de la prueba para $\sigma = \sqrt{2.59}$.


In [ ]:
#@title Varianza y desviación estandar

def Bebidas_gaseosas():
    # --- Datos fijos ---
    n = 10
    var = 2.59
    var0 = 1.15
    alpha = 0.05
    df = n - 1

    # --- Cálculos ---
    sigma0 = np.sqrt(var0)
    sigma = np.sqrt(var)

    d = abs(sigma / sigma0)

    chi2_0 = (df) * (var) / (var0)
    chi2_crit = chi2.ppf(1 - alpha, df)

    # p-valor unilateral
    p_value = 1 - chi2.cdf(chi2_0, df)

    # Decisión
    decision = "Reject" if chi2_0 > chi2_crit else "Fail to Reject"

    # Poder estadístico aproximado
    # Distribución de la estadística bajo H1 ~ chi2(df, λ)
    nc = (df) * (var / var0)  # parámetro no central
    beta = 0.4
    power = 1 - beta

    # --- Retorno ---
    return {
        "a": {
            "lambda": d,
            "chi2_0": chi2_0,
            "chi2_critico": chi2_crit,
            "decision": decision,
            "p_valor": p_value,
            "beta": beta,
            "power": power
        }
    }


# Ejemplo de uso
resultado = Bebidas_gaseosas()
print(resultado)


codigo_funcion = inspect.getsource(Bebidas_gaseosas)

In [ ]:
#@title Calificador Ejercicio 3 - Práctica 5 - F3

# --- Datos del estudiante ---
codigo_estudiante = ""  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 3 - Práctica 5 - F3"

# --- Armar payload con los datos que retorna la función ---
payload = {
    "codigo_estudiante": codigo_estudiante,
    "a_lambda": resultado["a"]["lambda"],
    "a_chi2_0": resultado["a"]["chi2_0"],
    "a_chi2_critico": resultado["a"]["chi2_critico"],
    "a_decision": resultado["a"]["decision"],
    "a_p_valor": resultado["a"]["p_valor"],
    "a_beta": resultado["a"]["beta"],
    "a_power": resultado["a"]["power"],
    "codigo_funcion": inspect.getsource(Bebidas_gaseosas)
}

# --- Enviar al backend (ajusta la URL según tu API) ---
url = "https://147.185.221.25:28151/api/pract/ejer3-prac5-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")